In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import datetime
from datetime import datetime
from astropy.coordinates import SkyCoord


In [2]:
import pandas as pd

In [3]:
%matplotlib inline

In [4]:
%matplotlib ipympl 

In [22]:
os.chdir(os.path.expanduser("~"))
print(os.getcwd())

/users/mseth


In [6]:
## DATA ##
# Load in data files

files = np.load('crab_files.npz', allow_pickle=True)

filepaths = files['crab_filepaths']
cascade_objs = files['crab_cascadeobj']

## Holography

In [25]:
sys.path.insert(0, os.path.abspath('beam-model'))
from beam_model import utils
from beam_model import formed 

In [26]:
print(sys.path[0])   

/users/mseth/beam-model


In [27]:
## DATA ##
# Get timestamps & indices for every peak 

max_timestamps = [] 
max_tidxs = []

for i, cascade_obj in enumerate(cascade_objs):
    ts = np.nansum(cascade_objs[i].beams[cascade_objs[i]._max_beam_idx].intensity, axis=0)
    max_time = np.argmax(ts)
    max_timestamp = cascade_objs[i].event_time + pd.Timedelta(seconds = max_time / 1000)
    
    max_timestamps.append(max_timestamp)
    max_tidxs.append(max_time)

In [28]:
## DATA ##
# Get HA (in CHIME) where each peak occurred

source_name = "CYG_A"
coords = SkyCoord.from_name(source_name)
source_ra = coords.ra.deg
source_dec = coords.dec.deg

has_at_peak = []
y_at_peak = []
for timestamp in max_timestamps:
    x, y = utils.get_position_from_equatorial(source_ra, source_dec, timestamp)
    has_at_peak.append(x)
    y_at_peak.append(y)

In [ ]:
freqs = np.linspace(400.390625, 800, 256)    # The data has 256 frequency channels

has = np.array(np.linspace(-105, 104.90278, 1024))

In [29]:
## BEAM ##
# Load in the beam response holography data 

path = "/users/mseth/holography_data/npz_files"

npz_files = []
for (root, dirs, file) in os.walk(path):
    for f in file: 
        npz_files.append(os.path.join(root, f))
xx_list = []
yy_list = []
for file in npz_files:
    data = np.load(file)
    xx = data['XX']
    yy = data['YY']
    xx_list.append(xx)
    yy_list.append(yy)

xx = np.vstack(xx_list)
yy = np.vstack(yy_list)
intensity = xx + yy / 2

# Get spectra of primary beam response at every HA 
int_responses = []

for ha in has_at_peak:
    idx = (np.abs(has - ha)).argmin()
    xx_response = xx[:,idx]
    yy_response = yy[:, idx]
    int_response = intensity[:,idx]
    int_responses.append(int_response)

NameError: name 'has' is not defined

In [ ]:
## DATA ## 
# Get spectra at every peak time 
spectra = []

for i, (cascade_obj, max_tidx) in enumerate(zip(cascade_objs, max_tidxs)):
    spectrum = cascade_objs[i].beams[cascade_objs[i]._max_beam_idx].intensity[:, max_tidx]
    spectra.append(spectrum)

spectra_at_peak = np.vstack(spectra)

In [ ]:
## DATA & BEAM RESPONSE ##

holo_corrected_sp = []

for i, (spectra, int_response) in enumerate(zip(spectra_at_peak, int_responses)):
    int_response_ds = int_response[::4]
    spectra_at_peak_corrected = spectra_at_peak / int_response_ds
    holo_corrected_sp.append(spectra_at_peak_corrected)

## Beamforming

In [122]:
# Load in beam model 
sbm = formed.FFTFormedBeamModel()

In [ ]:
def bf_to_jy(bf_spectrum, f_good):
    factor = (np.square(1024) * 128) / (np.square(4) * 0.806745 * 400)
    result = bf_spectrum / ( factor * np.square(f_good) ) 
    return result

In [ ]:
## DATA ##
# Get list of every beam id 

beam_ids = []
for cascade_obj in cascade_objs:
    beam_id = cascade_obj.beams[cascade_obj.__max_beam_id].beam_no
    beam_ids.append(beam_id)

In [ ]:
calibrated_spectra = []

for i, (spectrum, max_tidx, ha, y, beam_id) in enumerate(zip(holo_corrected_sp, has, y_at_peak, max_tidxs, beam_ids)):
    sensitivity = sbm.get_sensitivity([beam_id], np.array([ha, y_at_peak]).T, freqs).squeeze()
    
    mean_sensitivity = np.nanmean(sensitivity, axis=1)            #average over the spectra axis
    peak_sensitivity_ind = np.argmax(mean_sensitivity)            #find the peak of the mean sensitivity
    sensitivity_at_peak = sensitivity[peak_sensitivity_ind, :]    #Response at peak

    spectra_corrected = spectrum / sensitivity_at_peak
    spectra_jy = bf_to_jy(spectra_corrected, 1)
    calibrated_spectra.append(spectra_jy)

In [ ]:
for spectrum in calibrated_spectra:
    mean_flux = np.mean(spectrum)